<a href="https://colab.research.google.com/github/AmoreHH/data-course-sample/blob/main/W1/W1_rule_based_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-23 05:52:58--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  20.0MB/s    in 0.7s    

2021-12-23 05:52:59 (20.0 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-23 05:52:59--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.2MB/s    in 0.7s    

202

In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
ratings_trainings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25


## 產生推薦結果
使用以下4種方式產生推薦結果：

1. 以商品被評分次數高低，找出前10名熱門商品
2. 以商品所得評分高低，找出前10名高分商品
3. 超過100人以上評分商品才進行得分排序，找出前10名高分商品
4. 參考 IMDB formula，找出前10名加權高分商品








### 1. 以商品被評分次數高低找出前10名熱門商品

In [ ]:
def recommender_1(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    popularity = training_data["asin"].value_counts().index.to_list()[0:k]
    recommendations = {x: popularity for x in users}
    return recommendations

ratings_by_user_1 = recommender_1(ratings_trainings, users)

### 2. 以商品所得評分高低找出前10名高分商品

In [ ]:
def recommender_2(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    overall_m = training_data[["asin", "overall"]].groupby('asin').mean()
    overall_m = overall_m.sort_values(by='overall', ascending=False).index.to_list()[0:k]
    recommendations = {x: overall_m for x in users}
    return recommendations

ratings_by_user_2 = recommender_2(ratings_trainings, users)

### 3. 超過100人以上評分商品才進行得分排序，找出前10名高分商品

In [ ]:
popularity = ratings_trainings["asin"].value_counts()
popular_asin = popularity[popularity > 100].index
popular_asin

Index(['B000FOI48G', 'B000GLRREU', '1620213982', 'B001QY8QXM', 'B01DKQAXC0',
       'B00W259T7G', 'B006IB5T4W', 'B00005JS5C', 'B0012Y0ZG2', 'B000WYJTZG',
       ...
       'B00PMUTNOM', 'B001E5E9RK', 'B00NV6A3P2', 'B01GW09XRW', 'B000NJMSXU',
       'B00DIP4G4A', 'B01FY4WF54', 'B0014SQQ3M', 'B00J3WDBCK', 'B01BHM6K1C'],
      dtype='object', length=559)

In [ ]:
ratings_trainings_100 =  ratings_trainings[ratings_trainings["asin"].isin(popular_asin)]

In [ ]:
ratings_by_user_3 = recommender_2(ratings_trainings_100, users)

### 參考 IMDB formula，找出前10名加權高分商品
作法參考本篇：[Beginner Tutorial: Recommender Systems in Python](https://www.datacamp.com/community/tutorials/recommender-systems-python)

In [ ]:
review = ratings_trainings.groupby('asin').agg(review_count=('asin', 'count'), review_mean=('overall', 'mean')).reset_index()
review

,asin,review_count,review_mean
0,0061073717,2,5.000000
1,0143026860,17,4.117647
2,014789302X,20,4.350000
3,0571348351,3,5.000000
4,0692508988,1,5.000000
...,...,...,...
32577,B01HIWLLUK,1,5.000000
32578,B01HJ1K3YK,1,1.000000
32579,B01HJ84SGM,5,5.000000
32580,B01HJASD20,1,1.000000


In [ ]:
m = review["review_count"].quantile(0.90)
print(m)

C = review['review_mean'].mean()
print(C)


16.0
4.040947312571865


In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['review_count']
    R = x['review_mean']
    # refer to the IMDB formula (https://www.datacamp.com/community/tutorials/recommender-systems-python)
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
review['score'] = review.apply(weighted_rating, axis=1)
review

,asin,review_count,review_mean,score
0,0061073717,2,5.000000,4.147509
1,0143026860,17,4.117647,4.080459
2,014789302X,20,4.350000,4.212643
3,0571348351,3,5.000000,4.192377
4,0692508988,1,5.000000,4.097362
...,...,...,...,...
32577,B01HIWLLUK,1,5.000000,4.097362
32578,B01HJ1K3YK,1,1.000000,3.862068
32579,B01HJ84SGM,5,5.000000,4.269293
32580,B01HJASD20,1,1.000000,3.862068


In [ ]:
review = review.sort_values('score', ascending=False).reset_index(drop=True)
review_list=review['asin'].to_list()[0:10]

In [ ]:
ratings_by_user_4 = {x: review_list for x in users}

# 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

In [ ]:
method_1=evaluate(ratings_testings_by_user, ratings_by_user_1)
method_2=evaluate(ratings_testings_by_user, ratings_by_user_2)
method_3=evaluate(ratings_testings_by_user, ratings_by_user_3)
method_4=evaluate(ratings_testings_by_user, ratings_by_user_4)

result = {'method': ['1', '2', '3', '4'],
          'score' : [method_1, method_2, method_3, method_4]}

df = pd.DataFrame(result)
display (df)

,method,score
0,1,0.083051
1,2,0.000000
2,3,0.003390
3,4,0.000000
